In [1]:
import tensorflow as tf
import numpy as np
#from utils import build_model
import pandas as pd
from tqdm import tqdm
import imageio
import matplotlib.pyplot as plt
import keras
from keras.layers import Conv2D, Dropout, Flatten, Dense
from keras.regularizers import l2
import statsmodels.api as sm
from utils import build_model

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.4) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
checkpoint_path = '../../../../data/models/mc_dropout_cpl/export/'

In [3]:
model = build_model()
model.load_weights(tf.train.latest_checkpoint(checkpoint_path))

In [4]:
path_all_imgs = '../../../../data/commaai/training_files_filtered/indices/val_indices.csv'
all_img_df = pd.read_csv(path_all_imgs)
img_path_base = '../../../../data/commaai/train_bags_2/'
density_path = '../../../../data/commaai/density/gaussian_density_filtered.csv'
density = pd.read_csv(density_path)

In [5]:
# extract Bzetas by reading in images and predicting
labels = []
preds = []
for i in tqdm(range(0,all_img_df.shape[0])): 
    img = imageio.imread(str(img_path_base + all_img_df.loc[i,'path']))[:,:,0:3]/255
    pred = model.predict(img.reshape(1,66,200,3))
    label = all_img_df.loc[i,'angle']
    labels.append(label)       
    preds.append(pred)

# rearrange to arrays
labels = np.array(labels)
preds = np.array(preds)

100%|██████████| 10472/10472 [17:57<00:00,  9.72it/s]


In [6]:
np.save('preds.npy', preds)
np.save('labels.npy', labels)

In [ ]:
np.save('preds.npy', preds)
np.save('labels.npy', labels)

In [22]:
np.save('../../../../data/commaai/predictions/no_mc_cpl_preds.npy', preds)

## MC-Dropout

In [12]:
preds_samp = np.load('mc_preds.npy')
preds = np.load('preds.npy')
labels = np.load('labels.npy')

In [19]:
preds_1 = np.append(np.load('../../../../data/commaai/predictions/mc_dropout/mc_preds_2.npy'), 
                  np.load('../../../../data/commaai/predictions/mc_dropout/mc_preds_1.npy'), axis = 0)
preds_mc = np.mean(preds_1, axis = 1)

In [14]:
preds_mc = np.mean(preds_samp, axis = 1)

In [21]:
print("Performance of models without and with uncertainty" +
     "\nMSE:" +
      "\nwithout uncertainty: " + str(np.mean((labels - preds.reshape(10472,))**2)) +
      "\nwith uncertainty: " + str(np.mean((labels - preds_mc.reshape(10472,))**2)) +
      "\nMAE:" +
      "\nwithout uncertainty: " + str(np.mean(np.abs(labels - preds.reshape(10472,)))) +
      "\nwith uncertainty: " + str(np.mean(np.abs(labels - preds_mc.reshape(10472,)))) +
      "\nAccuracy :" +
      "\nwithout uncertainty: " + str(np.mean([np.abs((labels - preds.reshape(10472,))) <= 6])) +
      "\nwith uncertainty: " + str(np.mean([np.abs((labels - preds_mc.reshape(10472,))) <= 6])) +
      "\nAccuracy II:" +
      "\nwithout uncertainty: " + str(np.mean([np.abs((labels - preds.reshape(10472,))) <= 2])) +
      "\nwith uncertainty: " + str(np.mean([np.abs((labels - preds_mc.reshape(10472,))) <= 2])) +
      "\nAccuracy III:" +
      "\nwithout uncertainty: " + str(np.mean([np.abs((labels - preds.reshape(10472,))) <= 1])) +
      "\nwith uncertainty: " + str(np.mean([np.abs((labels - preds_mc.reshape(10472,))) <= 1])) +
      "\nAccuracy IV:" +
      "\nwithout uncertainty: " + str(np.mean([np.abs((labels - preds.reshape(10472,))) <= .5])) +
      "\nwith uncertainty: " + str(np.mean([np.abs((labels - preds_mc.reshape(10472,))) <= .5])) 
     )

Performance of models without and with uncertainty
MSE:
without uncertainty: 6.43083988934527
with uncertainty: 6.3471034626787475
MAE:
without uncertainty: 1.4190459962387845
with uncertainty: 1.4046055209861767
Accuracy :
without uncertainty: 0.9818563789152025
with uncertainty: 0.9836707410236822
Accuracy II:
without uncertainty: 0.7961229946524064
with uncertainty: 0.8008021390374331
Accuracy III:
without uncertainty: 0.5292207792207793
with uncertainty: 0.5313216195569137
Accuracy IV:
without uncertainty: 0.29258976317799845
with uncertainty: 0.29383116883116883


In [ ]:
imgs = []
labels = []
for i in tqdm(range(0,all_img_df.shape[0])): 
    img = imageio.imread(str(img_path_base + all_img_df.loc[i,'path']))[:,:,0:3]/255
    imgs.append(img)
    label = all_img_df.loc[i,'angle']
    labels.append(label) 
imgs = np.array(imgs)

In [ ]:
mc_samples = 1000
preds = []
for i in tqdm(range(0, imgs.shape[0])):
    x_pred = np.repeat(imgs[i, :,:,:], mc_samples, axis = 0)
    pred = model.predict(x_pred.reshape(-1,66,200,3))
    preds.append(pred)
preds = np.array(preds)

In [ ]:
np.save('mc_preds.npy', np.array(preds))

In [ ]:
preds = np.load('mc_preds.npy')

In [ ]:
preds = np.array(preds)

In [ ]:
mc_pred = np.mean(preds, axis = 1)

In [ ]:
import numpy as np

In [ ]:
np.mean(np.abs((labels - mc_pred)))

In [ ]:
preds.shape

In [ ]:
x_pred

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.scatter(labels, mc_pred, alpha = 0.1)

In [ ]:
for pred in preds:
    sns.kdeplot(np.array(pred).reshape(1000,), bw=0.5)

In [ ]:
# Estimate densities
kde = sm.nonparametric.KDEUnivariate(preds)
kde.fit() # Estimate the densities
support = kde.support
endog = kde.endog
density = kde.density

In [ ]:
plt.plot(support, density)